In [1]:

# from __future__ import division # un comment if you use python 2 !
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import time
import sys
import copy
#Quick fudge to make import from ../Scripts work

sys.path.append('../Scripts')


import gc


# import ClassCosmicIntegrator  as CI #Given settings and redshifts returns rates (2D arrays) Loads the data
from PostProcessingScripts import * 
import ClassCOMPAS     as CC ###
# import ClassFormationChannels as FC 
# from ClassFormationChannels_5mainchannels import * 

import pandas as pd
from astropy import units as u
from astropy import constants as const


dictDCOtypeDCOlabel = {'BBH':'BHBH', 'BNS':'NSNS', 'BHNS':'BHNS'}



from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



import h5py as h5
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import string


In [2]:
metallicityGrid =[0.0001, 0.00011, 0.00012, 0.00014, 0.00016, 0.00017,\
               0.00019, 0.00022, 0.00024, 0.00027, 0.0003, 0.00034, \
               0.00037, 0.00042, 0.00047, 0.00052, 0.00058, 0.00065,\
               0.00073, 0.00081, 0.0009, 0.00101, 0.00113, 0.00126,\
               0.0014, 0.00157, 0.00175, 0.00195, 0.00218, 0.00243, \
               0.00272, 0.00303, 0.00339, 0.00378, 0.00422, 0.00471, \
               0.00526, 0.00587, 0.00655, 0.00732, 0.00817, 0.00912, \
               0.01018, 0.01137, 0.01269, 0.01416, 0.01581, 0.01765, 0.01971, 0.022, 0.0244, 0.02705, 0.03]

In [3]:







def calculateEfficiencies(DCOtype='BBH', pathCOMPASOutput='/Volumes/Andromeda/DATA/AllDCO_bugfix/',BPS_model='A'):
    """ 

    DCOtype='BBH': type of compact object mergers, options: ['BHBH', 'BHNS' or 'NSNS'], 
    pathCOMPASOutput='/Volumes/Andromeda/DATA/AllDCO_bugfix/': path to directory with data 
    BPS_model='A' : alphabetical letter label of the data 
    
    calculates the formation efficiency of the requested DCOtype 
    
    returns an array of formation efficiency, each element in the array is the efficiency for a grid point in the metallicities. 

    """

#     BPSnameslist = list(string.ascii_uppercase)[0:nBPSmodels]   
    DCOname=dictDCOtypeDCOlabel[DCOtype]



    print('now at DCO type  ', DCOtype)
    # for ind_m, bps_model in enumerate(BPSnameslist):  	
    for ind_m, bps_model in enumerate(['A']):   
        print()
#         print('now at bps label,' bps_model)
        print('now at model ', alphabetDirDict[bps_model])

        # set always optimistic CE false, unless we are doing the optimistic variation
        OPTIMISTIC=False
        if (bps_model=='F') or (bps_model=='K'):
            OPTIMISTIC=True
            print('doing optimistic version of %s'%alphabetDirDict[bps_model])

        # path to datafile 
        # path = pathCOMPASOutput+alphabetDirDict[bps_model] + '/' + 'COMPASCompactOutput_'+DCOtype +'_'+bps_model+'.h5'
        path = pathCOMPASOutput+alphabetDirDict[bps_model] + '/' + 'COMPASOutput.h5'

        #But I want only within Hubble time 
        Data            = CC.COMPASData(path=path, lazyData=True, Mlower=5., \
                         Mupper=150, binaryFraction=1)
        Data.setCOMPASDCOmask(types=DCOtype,  withinHubbleTime=True, optimistic=OPTIMISTIC)
        Data.setCOMPASData()

        metallicities = Data.metallicitySystems
        seeds    = Data.seeds[Data.Hubble==True]
        weights = Data.weight

        # calculates the equivalent mass of the "Galaxy box" that is formed in stars, that represents our "COMPAS" box simulation (per Metallicity)
        Data_totalMassEvolvedPerZ = Data.totalMassEvolvedPerZ
        Data_metallicityGrid = Data.metallicityGrid
        del Data 



        listt=[0.0001, 0.00011, 0.00012, 0.00014, 0.00016, 0.00017,\
               0.00019, 0.00022, 0.00024, 0.00027, 0.0003, 0.00034, \
               0.00037, 0.00042, 0.00047, 0.00052, 0.00058, 0.00065,\
               0.00073, 0.00081, 0.0009, 0.00101, 0.00113, 0.00126,\
               0.0014, 0.00157, 0.00175, 0.00195, 0.00218, 0.00243, \
               0.00272, 0.00303, 0.00339, 0.00378, 0.00422, 0.00471, \
               0.00526, 0.00587, 0.00655, 0.00732, 0.00817, 0.00912, \
               0.01018, 0.01137, 0.01269, 0.01416, 0.01581, 0.01765, 0.01971, 0.022, 0.0244, 0.02705, 0.03]


        formationRateTotal           = np.zeros(len(listt))  


        # print('#Z =',len(Data.metallicityGrid))
        for nrZ, Z in enumerate(listt):
            # this if and else statement is a little hack. Data.metallicityGrid might not contains some metallicities since
            # it is based on the systems in the hdf5 file, but since the big Data files only contain the DCOs, it can be that a certain metallciity point
            # has 0 DCOs and thats what the data.metallicityGrid is based on         	
            if Z in Data_metallicityGrid:
                maskZ = (metallicities == Z)
                formationRateTotal[nrZ] = np.sum(weights[maskZ]) # //floor weights because not every binary in COMPAS is equally represented in the galaxy 
                # print('total 1 =',formationRateTotal[nrZ])


        # mask the Z that are in the grid	        
        maskZgridinZlist = np.in1d(listt, Data_metallicityGrid)

        formationRateTotal[maskZgridinZlist] = np.divide(formationRateTotal[maskZgridinZlist], Data_totalMassEvolvedPerZ) + 0 #lowerY        




    return formationRateTotal





In [ ]:
formationRateTotal_bps_model_A = calculateEfficiencies(DCOtype='BHNS', pathCOMPASOutput='/Volumes/Andromeda/DATA/AllDCO_bugfix/',BPS_model='A') 



In [ ]:
print('The formation efficiencies in units of solar masses formed is:')
print(formationRateTotal_bps_model_A, '[Msun^-1]' ) 
print()
print(len(formationRateTotal_bps_model_A)==len(metallicityGrid)) 
print('this array is the length of the metallicity grid')
print()
print('the corresponding metallicities are:')
print(metallicityGrid)
